# HW2

## 2.1 Deep CNN 

kernel sizes is (3, 3), strides is (1, 1), padding is same, network depth.

In [223]:
from keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D
from keras.models import Model
from keras.models import Sequential
from keras import backend as K
import keras
model = Sequential()
num_classes = 10
input_shape = (28, 28, 1)
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same',
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [224]:
from keras.datasets import fashion_mnist
import numpy as np

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (-1, 28, 28, 1))
x_test = np.reshape(x_test, (-1, 28, 28, 1))

In [225]:
print('X_train shape:', x_train.shape)
from keras.utils import np_utils
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('Y_train shape:', y_train.shape)

X_train shape: (60000, 28, 28, 1)
Y_train shape: (60000, 10)


In [226]:

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
model.fit(x_train, y_train,
                epochs=3,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, y_test),
                )

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 80s 1ms/step - loss: 0.5247 - acc: 0.8084 - val_loss: 0.3779 - val_acc: 0.8592
Epoch 2/3
60000/60000 [==============================] - 46s 763us/step - loss: 0.3103 - acc: 0.8862 - val_loss: 0.2981 - val_acc: 0.8914
Epoch 3/3
60000/60000 [==============================] - 44s 727us/step - loss: 0.2598 - acc: 0.9047 - val_loss: 0.3053 - val_acc: 0.8890


In [229]:
train_score = model.evaluate(x_train, y_train)
test_score = model.evaluate(x_test, y_test)


10000/10000 [==============================] - 3s 344us/step


In [232]:
print ("train loss is "+ str(train_score[0]) )
print ("train accuracy is "+ str(train_score[1]) )
print ("test loss is "+ str(test_score[0]) )
print ("test accuracy is "+ str(test_score[1]) )

train loss is 0.2542090878779689
train accuracy is 0.90535
test loss is 0.30529952994585036
test accuracy is 0.889


## 2.2 Transfer Learning

In [233]:
import numpy as np

In [234]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [235]:
x_train = [cv2.cvtColor(cv2.resize(i, (48,48)), cv2.COLOR_GRAY2BGR) for i in x_train]
x_train = np.concatenate([arr[np.newaxis] for arr in x_train]).astype('float32')
x_test = [cv2.cvtColor(cv2.resize(i, (48,48)), cv2.COLOR_GRAY2BGR) for i in x_test]
x_test = np.concatenate([arr[np.newaxis] for arr in x_test]).astype('float32')

In [236]:
from keras.applications.resnet50 import ResNet50
base = ResNet50(weights='imagenet', include_top=False, input_shape=(48,48,3))
for layer in base.layers:
    layer.trainable = False

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [240]:
model2 = Sequential()
model2.add(base)
model2.add(Dense(1000, activation='relu'))
model2.add(Flatten())
model2.add(Dense(num_classes, activation='softmax'))
model2.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
print (x_train.shape)
print (x_test.shape)
model2.fit(x_train, y_train,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, y_test),
                )

(60000, 48, 48, 3)
(10000, 48, 48, 3)
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 394s 7ms/step - loss: 0.6326 - acc: 0.8068 - val_loss: 1.2922 - val_acc: 0.5743
Epoch 2/5
60000/60000 [==============================] - 333s 6ms/step - loss: 0.2604 - acc: 0.9065 - val_loss: 1.6214 - val_acc: 0.5212
Epoch 3/5
60000/60000 [==============================] - 328s 5ms/step - loss: 0.1407 - acc: 0.9514 - val_loss: 1.4039 - val_acc: 0.5822
Epoch 4/5
60000/60000 [==============================] - 330s 5ms/step - loss: 0.0726 - acc: 0.9770 - val_loss: 1.6285 - val_acc: 0.5782
Epoch 5/5
60000/60000 [==============================] - 328s 5ms/step - loss: 0.0371 - acc: 0.9895 - val_loss: 1.7379 - val_acc: 0.5854


In [243]:
train_score = model2.evaluate(x_train, y_train)
test_score = model2.evaluate(x_test, y_test)

10000/10000 [==============================] - 63s 6ms/step


In [244]:
print ("train loss is "+ str(train_score[0]) )
print ("train accuracy is "+ str(train_score[1]) )
print ("test loss is "+ str(test_score[0]) )
print ("test accuracy is "+ str(test_score[1]) )

train loss is 1.7187981756210327
train accuracy is 0.5842
test loss is 1.7378805902481078
test accuracy is 0.5854
